# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [40]:
import pandas as pd
import numpy as mp
from scipy.spatial.distance import pdist, squareform

In [41]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [42]:
display(data.head())
data.shape

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [43]:
data2 = data.groupby(["CustomerID", "ProductName"])["Quantity"].sum().reset_index()
display(data2.head())

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [46]:
data_pivot = pd.pivot_table(data, values = "Quantity", index=["ProductName"], columns = ["CustomerID"], aggfunc=sum)
data_pivot.fillna(0, inplace=True)
display(data_pivot.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [47]:
# https://en.wikipedia.org/wiki/Euclidean_distance
similarity_m = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)
display(similarity_m.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [59]:
similarities = similarity_m[200].sort_values(ascending=False)[1:6]
display(similarities.head(5))
similar_ids=list(similarities.index)

print(similar_ids)

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

[3317, 1072, 3535, 1920, 3909]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [72]:
records=data2.loc[data2['CustomerID'].isin(similar_ids)]
display(records)

,CustomerID,ProductName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1083,1072,Appetizer - Sausage Rolls,1
1084,1072,"Beans - Kidney, Canned",1
1085,1072,Beef - Montreal Smoked Brisket,1
1086,1072,Beef - Striploin Aa,1
...,...,...,...
3263,3909,Wine - Charddonnay Errazuriz,2
3264,3909,Wine - Fume Blanc Fetzer,1
3265,3909,"Wine - Magnotta, Merlot Sr Vqa",1
3266,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [92]:
top_products = records.groupby(["ProductName"])["Quantity"].sum().reset_index()
top_products = top_products.sort_values("Quantity",ascending=False)
display(top_products.head(20))

,ProductName,Quantity
159,Soup - Campbells Bean Medley,4
103,Muffin - Carrot Individual Wrap,3
8,Bay Leaf,3
136,Pork - Kidney,3
129,"Pepper - Black, Whole",3
88,Lamb - Ground,3
185,Wanton Wrap,3
48,Coffee - Irish Cream,2
173,"Tart Shells - Sweet, 4",2
172,Tahini Paste,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [102]:
##records of cid=200
records_bought=data2.loc[data2['CustomerID']==200]
##creating list with unique products bought
products_bought=list(records_bought.ProductName.unique())
print(len(products_bought))
##print(products_bought)

##creating list with unique products bought by similar customers
products_similar=list(top_products.ProductName.unique())
print(len(products_similar))
##print(products_similar)


## creating list bought by similar customers not bought already by customer 200
products_to_recommend=[i for i in products_similar if i not in products_bought]
print(len(products_to_recommend))
print(products_to_recommend)
        

72
207
165
['Soup - Campbells Bean Medley', 'Muffin - Carrot Individual Wrap', 'Bay Leaf', 'Pork - Kidney', 'Wanton Wrap', 'Chips Potato Salt Vinegar 43g', 'Table Cloth 54x72 White', 'Chinese Foods - Chicken', 'Lime Cordial - Roses', 'Soupfoamcont12oz 112con', 'Lamb - Pieces, Diced', 'Cheese Cloth No 100', 'Dried Figs', 'Eggplant - Asian', 'Soup - Campbells, Beef Barley', 'Ezy Change Mophandle', 'Fish - Scallops, Cold Smoked', 'Foam Dinner Plate', 'Fondant - Icing', 'Garbage Bags - Clear', 'Sole - Dover, Whole, Fresh', 'Sausage - Liver', 'Scallops - 10/20', 'Ice Cream Bar - Hageen Daz To', 'Ice Cream Bar - Oreo Cone', 'Juice - Orange', 'Scallops 60/80 Iqf', 'Scampi Tail', 'Black Currants', 'Wine - Toasted Head', 'Wine - Magnotta, Merlot Sr Vqa', 'Pate - Cognac', 'Wine - Chardonnay South', 'Wine - Charddonnay Errazuriz', 'Beef Wellington', 'Wine - Blue Nun Qualitatswein', 'Wine - Red, Harrow Estates, Cab', 'Beans - Kidney, Canned', 'Bread - Rye', 'Muffin Batt - Blueberry Passion', 'Past

In [107]:
top5=top_products.loc[top_products['ProductName'].isin(products_to_recommend)][:5]
print(top5['ProductName'])

159       Soup - Campbells Bean Medley
103    Muffin - Carrot Individual Wrap
8                             Bay Leaf
136                      Pork - Kidney
185                        Wanton Wrap
Name: ProductName, dtype: object


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.